# Flyweight Pattern

Uses sharing to support large numbers of fine-grained objects efficiently.

## Intent

- Minimize memory usage by sharing common parts of state between objects
- Support large numbers of similar objects efficiently
- Separate intrinsic (shared) from extrinsic (unique) state
- Maintain performance when object quantity increases dramatically

## Implementation 1: Basic Flyweight Pattern

In [1]:
from typing import Dict


# Flyweight class - contains intrinsic state shared across objects
class Flyweight:
    def __init__(self, shared_state):
        self._shared_state = shared_state

    def operation(self, unique_state):
        """Operation using both shared (intrinsic) and unique (extrinsic) state"""
        shared = ", ".join(str(item) for item in self._shared_state)
        unique = ", ".join(str(item) for item in unique_state)
        return f"Flyweight: Displaying shared state ({shared}) and unique state ({unique})"


# Factory that creates and manages flyweight objects
class FlyweightFactory:
    _flyweights: Dict[str, Flyweight] = {}

    def __init__(self, initial_flyweights):
        # Initialize with a set of flyweights
        for state in initial_flyweights:
            self.get_flyweight(state)

    def get_flyweight(self, shared_state) -> Flyweight:
        """Returns an existing flyweight with the specified shared state or creates a new one"""
        # Convert shared state into a string key
        key = self._get_key(shared_state)

        # If this flyweight doesn't exist, create it
        if key not in self._flyweights:
            print(f"FlyweightFactory: Creating new flyweight with state {shared_state}")
            self._flyweights[key] = Flyweight(shared_state)
        else:
            print(f"FlyweightFactory: Reusing existing flyweight with state {shared_state}")

        return self._flyweights[key]

    def list_flyweights(self):
        """Lists all flyweights and their keys"""
        count = len(self._flyweights)
        print(f"\nFlyweightFactory: I have {count} flyweights:")
        for key in self._flyweights.keys():
            print(key)

    def _get_key(self, state) -> str:
        """Create a hash key from shared state"""
        # Sort state to ensure consistent key generation
        return "_".join(sorted(str(item) for item in state))

### Usage

In [2]:
def add_car_to_database(factory, license_plate, owner, brand, model, color):
    print("\nClient: Adding a car to database.")

    # Shared (intrinsic) state: brand, model, color
    flyweight = factory.get_flyweight([brand, model, color])

    # Unique (extrinsic) state: license plate, owner
    result = flyweight.operation([license_plate, owner])
    print(result)


if __name__ == "__main__":
    # Create factory with initial flyweights
    factory = FlyweightFactory(
        [
            ["Chevrolet", "Camaro", "red"],
            ["Mercedes Benz", "C300", "black"],
            ["Mercedes Benz", "C500", "red"],
            ["BMW", "M5", "red"],
            ["BMW", "X6", "white"],
        ]
    )

    factory.list_flyweights()

    # Add cars using the factory - notice reuse of flyweights
    add_car_to_database(factory, "CL234IR", "James Doe", "BMW", "M5", "red")
    add_car_to_database(factory, "CL567IR", "Alice Smith", "BMW", "X1", "blue")

    factory.list_flyweights()

FlyweightFactory: Creating new flyweight with state ['Chevrolet', 'Camaro', 'red']
FlyweightFactory: Creating new flyweight with state ['Mercedes Benz', 'C300', 'black']
FlyweightFactory: Creating new flyweight with state ['Mercedes Benz', 'C500', 'red']
FlyweightFactory: Creating new flyweight with state ['BMW', 'M5', 'red']
FlyweightFactory: Creating new flyweight with state ['BMW', 'X6', 'white']

FlyweightFactory: I have 5 flyweights:
Camaro_Chevrolet_red
C300_Mercedes Benz_black
C500_Mercedes Benz_red
BMW_M5_red
BMW_X6_white

Client: Adding a car to database.
FlyweightFactory: Reusing existing flyweight with state ['BMW', 'M5', 'red']
Flyweight: Displaying shared state (BMW, M5, red) and unique state (CL234IR, James Doe)

Client: Adding a car to database.
FlyweightFactory: Creating new flyweight with state ['BMW', 'X1', 'blue']
Flyweight: Displaying shared state (BMW, X1, blue) and unique state (CL567IR, Alice Smith)

FlyweightFactory: I have 6 flyweights:
Camaro_Chevrolet_red
C30

## Implementation 2: Text Formatter Example

In [3]:
class CharacterStyle:
    """Flyweight - stores character formatting (shared state)"""

    def __init__(self, font_name, font_size, bold=False, italic=False):
        self.font_name = font_name
        self.font_size = font_size
        self.bold = bold
        self.italic = italic

    def __str__(self):
        style = []
        style.append(f"Font: {self.font_name}")
        style.append(f"Size: {self.font_size}")
        if self.bold:
            style.append("Bold")
        if self.italic:
            style.append("Italic")
        return ", ".join(style)


class StyleFactory:
    """Factory that creates and manages character style flyweights"""

    _styles = {}

    @classmethod
    def get_style(cls, font_name, font_size, bold=False, italic=False):
        # Create a key for this style combination
        key = (font_name, font_size, bold, italic)

        # Return existing style or create a new one
        if key not in cls._styles:
            cls._styles[key] = CharacterStyle(font_name, font_size, bold, italic)
            print(f"Created new style: {cls._styles[key]}")
        else:
            print(f"Reusing style: {cls._styles[key]}")

        return cls._styles[key]

    @classmethod
    def get_style_count(cls):
        return len(cls._styles)


class Character:
    """Context class that uses flyweight for styling"""

    def __init__(self, char, style):
        self.char = char  # Extrinsic state (unique per character)
        self.style = style  # Intrinsic state (shared style)

    def __str__(self):
        return f"{self.char} ({self.style})"


class TextEditor:
    def __init__(self):
        self.characters = []

    def add_character(self, char, font_name, font_size, bold=False, italic=False):
        style = StyleFactory.get_style(font_name, font_size, bold, italic)
        self.characters.append(Character(char, style))

    def __str__(self):
        return "".join(char.char for char in self.characters)

### Usage

In [4]:
if __name__ == "__main__":
    editor = TextEditor()

    # Add text with various styles
    editor.add_character("H", "Arial", 12, True, False)
    editor.add_character("e", "Arial", 12, True, False)
    editor.add_character("l", "Arial", 12, True, False)
    editor.add_character("l", "Arial", 12, True, False)  # Reuse existing style
    editor.add_character("o", "Arial", 12, True, False)
    editor.add_character(" ", "Arial", 12, False, False)
    editor.add_character("W", "Times New Roman", 14, False, True)
    editor.add_character("o", "Times New Roman", 14, False, True)
    editor.add_character("r", "Times New Roman", 14, False, True)
    editor.add_character("l", "Times New Roman", 14, False, True)
    editor.add_character("d", "Times New Roman", 14, False, True)
    editor.add_character("!", "Arial", 12, True, False)

    print(f"\nText: {editor}")
    print(f"Total characters: {len(editor.characters)}")
    print(f"Total styles (flyweights): {StyleFactory.get_style_count()}")

Created new style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Created new style: Font: Arial, Size: 12
Created new style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Arial, Size: 12, Bold

Text: Hello World!
Total characters: 12
Total styles (flyweights): 3
